# **Machine Learning**

# Taxi Fare Prediction using Spark ML

In this notebook, we will explore the task of predicting taxi fare amounts based on various features related to taxi trips in New York City. We will utilize the power of Apache Spark and its Machine Learning library, Spark ML, to build and evaluate predictive models.

## Dataset Overview

The dataset we are working with contains a wealth of information about taxi trips, including pick-up and drop-off times, passenger counts, trip distances, fare amounts, and more. Here's a brief overview of the dataset:

- **VendorID:** A code indicating the TPEP provider.
- **tpep_pickup_datetime:** The time when the trip started.
- **tpep_dropoff_datetime:** The time when the trip ended.
- **Passenger_count:** The number of passengers in the taxi.
- **Trip_distance:** The distance of the trip in miles.
- **RateCodeID:** The rate code for the trip (e.g., standard rate, airport fare).
- **Other Features:** There are additional features such as `Hour_of_Pickup`, `Day_of_Week`, `trip_duration`, and more, which we will engineer for our predictive models.

## Objective

Our primary goal is to build machine learning models that can accurately predict the total fare amount for taxi trips. We will explore different algorithms and hyperparameter tuning techniques to achieve the best predictive performance.

## Approach

1. **Data Preprocessing:** We will start by loading and preprocessing the dataset, including handling missing values, encoding categorical features, and creating new features.
   
2. **Feature Selection:** We will analyze the dataset to select the most relevant features for our prediction task.
   
3. **Model Building:** We will experiment with multiple machine learning algorithms, including Random Forest, Gradient Boosting, and more.
   
4. **Hyperparameter Tuning:** We will fine-tune the hyperparameters of our models using techniques like grid search or random search.
   
5. **Model Evaluation:** We will evaluate the models using appropriate evaluation metrics such as Root Mean Squared Error (RMSE), Mean Absolute Error (MAE), and R-squared (R2).

6. **Prediction:** Finally, we will use the best-performing model to make predictions on a test dataset and assess its performance.

## Environment Setup

Make sure you have set up your Spark environment with the necessary configurations and libraries. You can install required libraries using `pip` or `conda`.

Let's get started with data loading and preprocessing!


# 1. Data Loading

In [0]:
combined_data = spark.read.parquet("/finalds/combined/1.parquet")
combined_data.count()




37411142

Cache the dataframe `combined_data` using `.cache()`:

In [0]:
combined_data.cache()

DataFrame[VendorID: bigint, lpep_pickup_datetime: timestamp, lpep_dropoff_datetime: timestamp, store_and_fwd_flag: string, RatecodeID: double, PULocationID: bigint, DOLocationID: bigint, passenger_count: double, trip_distance: double, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, ehail_fee: double, improvement_surcharge: double, total_amount: double, payment_type: double, trip_type: double, congestion_surcharge: double]

Print the schema of `combined_data` using printSchema():

In [0]:
combined_data.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



Print the number of rows and columns of `combined_data`

# 2. Data Preparation

## Data Preprocessing: Filtering and Feature Engineering

In this section, we perform essential data preprocessing tasks on our taxi trip dataset. These tasks include filtering the data for the years 2021 and 2022 and creating new features that will be used for our predictive modeling. Let's break down what we're doing:

1. **Filtering Data for 2021 and 2022:** We start by filtering our dataset to include only taxi trips that occurred in the years 2021 and 2022. This step helps us focus on recent data for our analysis and model training.

2. **Feature Engineering:** We engineer several features that are relevant for our prediction task:
   - `Hour_of_Pickup`: We extract the hour component from the `tpep_pickup_datetime` column to understand the time of day when the trip started.
   - `Day_of_Week`: We extract the day of the week from the `tpep_pickup_datetime` column to consider the day of the week's influence on fares.
   - `trip_duration`: We calculate the trip duration in hours by computing the time difference between `tpep_pickup_datetime` and `tpep_dropoff_datetime` and converting it from seconds to hours.
   - `Passenger_count`: We retain the passenger count as a feature.
   - `trip_distance`: We keep the trip distance as a feature.
   - `RateCodeID`: We retain the rate code, which represents the fare structure, as a feature.
   - `total_amount`: We keep the total fare amount as our target variable for prediction.
   - `tpep_pickup_datetime` and `tpep_dropoff_datetime`: We retain these columns for reference.

3. **Data Saving:** After preprocessing and feature engineering, we save the filtered and transformed data as a new Parquet file for further analysis and model building.

Now, let's proceed with the code to perform these preprocessing steps.


In [0]:
from pyspark.sql.functions import hour, dayofweek, expr

# Assuming you have already loaded the combined_data DataFrame
combined_data.createOrReplaceTempView("taxi_data")

# Use Spark SQL to filter data for 2021 and 2022 and select the desired columns
filtered_data = spark.sql("""
    SELECT
        hour(tpep_pickup_datetime) AS Hour_of_Pickup,
        dayofweek(tpep_pickup_datetime) AS Day_of_Week,
        (unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 3600 AS trip_duration,  -- Convert seconds to hours for trip duration
        Passenger_count,
        trip_distance,
        RateCodeID,
        total_amount, 
        tpep_pickup_datetime, 
        tpep_dropoff_datetime
    FROM
        taxi_data
    
""")

# Show a sample of the filtered data
filtered_data.show()

# Save the filtered data as a new Parquet file
filtered_data.write.mode("overwrite").parquet("/FilteredData/filtered_taxi_data.parquet")

# To check the number of rows in the filtered data
filtered_data_count = filtered_data.count()
print("Number of rows in the filtered data:", filtered_data_count)


+--------------+-----------+-------------------+---------------+-------------+----------+------------+--------------------+---------------------+
|Hour_of_Pickup|Day_of_Week|      trip_duration|Passenger_count|trip_distance|RateCodeID|total_amount|tpep_pickup_datetime|tpep_dropoff_datetime|
+--------------+-----------+-------------------+---------------+-------------+----------+------------+--------------------+---------------------+
|            23|          7| 0.3338888888888889|            1.0|          4.7|       1.0|       26.16| 2022-11-26 23:55:08|  2022-11-27 00:15:10|
|            23|          7|0.17472222222222222|            1.0|         6.19|       1.0|        19.3| 2022-11-26 23:00:44|  2022-11-26 23:11:13|
|            23|          7|               0.11|            2.0|          1.2|       1.0|       12.85| 2022-11-26 23:19:56|  2022-11-26 23:26:32|
|            23|          7|             0.0725|            1.0|          0.6|       1.0|         8.3| 2022-11-26 23:34:11| 

## Data Splitting: Training and Test Sets

In this section, we split our filtered and preprocessed data into two distinct sets: a training dataset and a test dataset. The purpose of this split is to reserve a portion of the data for evaluating the performance of our predictive models. Here's what we're doing:

1. **Filtering Data for Training and Test:** We divide the data into two time periods:
   - **Training Dataset (January to September):** We select taxi trips that occurred between January and September, which will be used for training our predictive models.
   - **Test Dataset (October to December):** We choose taxi trips from October to December, which will serve as our test dataset for model evaluation.

2. **Data Saving:** After splitting the data, we save both the training and test datasets as separate Parquet files. This allows us to load and use these datasets for training and evaluating machine learning models.

3. **Data Count:** We also provide information about the number of rows in each dataset for reference.

This splitting ensures that our model is trained on historical data and evaluated on more recent data, helping us assess its generalization performance.


In [0]:
from pyspark.sql.functions import month

# Filter data for training dataset (January to September)
train_data = filtered_data.filter((month(filtered_data["tpep_pickup_datetime"]) >= 1) & (month(filtered_data["tpep_pickup_datetime"]) <= 9))

# Filter data for test dataset (October to December)
test_data = filtered_data.filter((month(filtered_data["tpep_pickup_datetime"]) >= 10) & (month(filtered_data["tpep_pickup_datetime"]) <= 12))

# Save the filtered training and test data as Parquet files
train_data.write.mode("overwrite").parquet("/FilteredData/train_taxi_data.parquet")
test_data.write.mode("overwrite").parquet("/FilteredData/test_taxi_data.parquet")

# To check the number of rows in each dataset
train_data_count = train_data.count()
test_data_count = test_data.count()
print("Number of rows in the training data:", train_data_count)
print("Number of rows in the test data:", test_data_count)


Number of rows in the training data: 27697639
Number of rows in the test data: 9713503


Copy the dataframe into a new variable called `df_features`

In [0]:
                            
train_data= spark.read.parquet("/FilteredData/train_taxi_data.parquet")
test_data=spark.read.parquet("/FilteredData/test_taxi_data.parquet")


In [0]:
filtered_data.printSchema()

root
 |-- Hour_of_Pickup: integer (nullable = true)
 |-- Day_of_Week: integer (nullable = true)
 |-- trip_duration: double (nullable = true)
 |-- Passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RateCodeID: double (nullable = true)
 |-- total_amount: double (nullable = true)



In the schema, it seems that we have correctly chosen integer for categorical features (Hour_of_Pickup, Day_of_Week, Passenger_count, RateCodeID) and double for continuous features (trip_duration, trip_distance, total_amount), which is a reasonable choice based on the nature of these features.

## Feature Correlation Analysis

In this section, we perform a correlation analysis to understand the relationships between our selected features and the target variable, `Total_amount`. Here's what we're doing:

1. **Selecting Features:** We have chosen a set of features, including `Hour_of_Pickup`, `Day_of_Week`, `trip_duration`, `Passenger_count`, `trip_distance`, and `RateCodeID`, to analyze their correlation with `Total_amount`. These features were intuitively selected based on our understanding of taxi fare determinants.

2. **Calculating Correlation Coefficients:** For each selected feature, we calculate the correlation coefficient with `Total_amount`. The correlation coefficient measures the strength and direction of the linear relationship between two variables.

3. **Storing Results:** We store the computed correlation coefficients in a dictionary called `correlation_coefficients`.

4. **Printing Results:** Finally, we print the correlation coefficients to understand how each feature is related to the total fare amount. Positive values indicate a positive correlation, while negative values suggest a negative correlation. A value closer to 1 or -1 indicates a stronger relationship, while values close to 0 suggest a weaker relationship.

Our initial selection of these features was guided by common intuitions about taxi fare determinants:

- `Hour_of_Pickup`: We anticipated that the time of day when a taxi trip starts could influence fares, with potential differences during rush hours or late at night.

- `Day_of_Week`: We considered that fares might vary based on the day of the week, especially on weekends or holidays.

- `trip_duration`: Longer trips often incur higher fares, so we included this as a potential predictor.

- `Passenger_count`: Different taxi services may have varying pricing structures based on passenger count.

- `trip_distance`: The distance traveled is a fundamental factor in fare calculation, and we expected a strong correlation.

- `RateCodeID`: We included this as it represents different fare structures, such as standard rates, airport fares, etc.

This analysis helps us identify which features may have a significant impact on predicting taxi fare amounts and guides feature selection for our machine learning models.



In [0]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

# List of features to calculate correlation with Total_amount
features = [
    "Hour_of_Pickup",
    "Day_of_Week",
    "trip_duration",
    "Passenger_count",
    "trip_distance",
    "RateCodeID"
]

# Create an empty dictionary to store the correlation coefficients
correlation_coefficients = {}

# Iterate through the features and calculate correlation with Total_amount
for feature in features:
    # Assemble the feature and Total_amount into a vector
    vector_assembler = VectorAssembler(inputCols=[feature, "total_amount"], outputCol="features")
    assembled_data = vector_assembler.transform(train_data)

    # Calculate the correlation matrix
    corr_matrix = Correlation.corr(assembled_data, "features")

    # Extract the correlation coefficient between the feature and Total_amount
    correlation_coefficient = corr_matrix.collect()[0][0]

    # Store the correlation coefficient in the dictionary
    correlation_coefficients[feature] = correlation_coefficient

# Print the correlation coefficients
for feature, coefficient in correlation_coefficients.items():
    print(f"Correlation between {feature} and Total_amount: {coefficient}")


Correlation between Hour_of_Pickup and Total_amount: DenseMatrix([[1.00000000e+00, 1.01103732e-04],
             [1.01103732e-04, 1.00000000e+00]])
Correlation between Day_of_Week and Total_amount: DenseMatrix([[ 1.       , -0.0029578],
             [-0.0029578,  1.       ]])
Correlation between trip_duration and Total_amount: DenseMatrix([[1.        , 0.03141544],
             [0.03141544, 1.        ]])
Correlation between Passenger_count and Total_amount: DenseMatrix([[1.        , 0.00520525],
             [0.00520525, 1.        ]])
Correlation between trip_distance and Total_amount: DenseMatrix([[1.        , 0.13896552],
             [0.13896552, 1.        ]])
Correlation between RateCodeID and Total_amount: DenseMatrix([[1.        , 0.09122573],
             [0.09122573, 1.        ]])


**Interpreting Correlation Results:**

1. The correlation coefficient between `Hour_of_Pickup` and `Total_amount` is approximately 0.0001, indicating an extremely weak positive correlation. This suggests that the time of day when a trip starts has minimal impact on the total fare amount.

2. For `Day_of_Week` and `Total_amount`, the correlation coefficient is close to -0.003, also indicating a very weak correlation. Day of the week seems to have a negligible influence on taxi fares.

3. `trip_duration` exhibits a slightly stronger positive correlation of approximately 0.0314 with `Total_amount`. Longer trip durations are associated with higher fares, but the correlation is still relatively weak.

4. `Passenger_count` has a correlation coefficient of about 0.0052 with `Total_amount`, indicating a very weak positive correlation. The number of passengers in a taxi appears to have a minimal impact on fare.

5. The `trip_distance` feature shows a moderate positive correlation with `Total_amount`, with a coefficient of approximately 0.139. This suggests that fare amounts tend to increase as the distance traveled becomes greater.

6. `RateCodeID` demonstrates a somewhat stronger positive correlation with `Total_amount`, around 0.0912. Different rate codes, which represent fare structures, can influence taxi fares to some extent.

While some features exhibit weak correlations with the total fare amount, it's essential to note that correlation does not imply causation. Further analysis and machine learning modeling will help us better understand the predictive power of these features in estimating taxi fares.
**Next Steps:**
With the said, it's still prudent to check if the performance of the model improves with the combination of these features, that might give us some information about the data. 


## Next Steps

To gain further insights into the data and assess the potential impact of feature combinations, we will embark on a multi-step modeling journey. Our objective is to determine if incorporating these selected features enhances predictive accuracy. Here's our roadmap:
## Predictive Modeling Approach

In our predictive modeling process, we follow a step-by-step approach to build and assess the performance of different models for predicting total fare amounts. The primary goal is to enhance the accuracy of fare predictions and gain insights into the underlying data patterns. The key steps in our modeling approach are as follows:

1. **Baseline Model:** We start with a baseline model, which is a fundamental approach to set a benchmark for our predictions. In this step, we calculate the average paid amount per trip in the training dataset. This straightforward baseline model provides a baseline metric for comparison.

2. **Linear Regression:** After establishing the baseline, we employ Linear Regression, a widely-used regression technique, to build our predictive model. Linear Regression helps us explore the linear relationships between input features and total fare amounts. We assess whether this linear model can outperform our baseline model and provide more accurate predictions.

3. **Random Forest Regression:** Following the LR assessment, we leverage the Random Forest Regressor, a robust machine learning algorithm, to build a predictive model. By doing so, we aim to investigate whether this ensemble method can outperform our baseline.

These sequential steps help us gauge the effectiveness of our feature combination strategy and provide valuable insights into how well our models capture the underlying patterns in the data. Ultimately, this process aids in making informed decisions about feature selection and model selection for more accurate predictions.


In [0]:
# Assuming you have already loaded the filtered_data DataFrame
unique_passenger_counts = filtered_data.select("Passenger_count").distinct()

# Show the unique passenger counts
unique_passenger_counts.show()


+---------------+
|Passenger_count|
+---------------+
|            7.0|
|            1.0|
|            4.0|
|            3.0|
|            2.0|
|            6.0|
|            5.0|
+---------------+



#Baseline Model

In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
import pyspark.sql.functions as F

# Calculate the average paid amount per trip from Part 2 Q3
avg_amount = 43.47990516867878

# Add a new column with the average amount as the prediction
baseline_predictions = test_data.withColumn("prediction", F.lit(avg_amount))

# Evaluate the performance of the baseline model
evaluator = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction", metricName="rmse")
baseline_rmse = evaluator.evaluate(baseline_predictions)
print("RMSE for Average Amount Model:", baseline_rmse)


RMSE for Average Amount Model: 111.16547072874559


#Linear Regression

In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

# Define your features and create a vector assembler
features = [
    "Hour_of_Pickup",
    "Day_of_Week",
    "trip_duration",
    "Passenger_count",
    "trip_distance",
    "RateCodeID"
]

vector_assembler = VectorAssembler(inputCols=features, outputCol="features")

# Define the Linear Regression model
lr = LinearRegression(featuresCol="features", labelCol="total_amount")

# Create a pipeline with the vector assembler and Linear Regression
pipeline = Pipeline(stages=[vector_assembler, lr])

# Fit the model on the training data
model = pipeline.fit(train_data)

# Make predictions on the test data
predictions = model.transform(test_data)

# Evaluate the model using a regression evaluator
evaluator = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

# Print the RMSE
print("Root Mean Squared Error (RMSE):", rmse)


Root Mean Squared Error (RMSE): 7.736532279687777


In [0]:
# Make predictions on the test data
predictions = model.transform(train_data)

# Evaluate the model using a regression evaluator
evaluator = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

# Print the RMSE
print("Root Mean Squared Error (RMSE) on Train Data:", rmse)

Root Mean Squared Error (RMSE) on Train Data: 107.7431383362781


#Random Forest Model

In [0]:
from pyspark.ml import Pipeline  
from pyspark.ml.regression import RandomForestRegressor

# Define the features and create a vector assembler
features = [
    "Hour_of_Pickup",
    "Day_of_Week",
    "trip_duration",
    "Passenger_count",
    "trip_distance",
    "RateCodeID"
]
vector_assembler = VectorAssembler(inputCols=features, outputCol="features")

# Define the Random Forest Regressor
rf = RandomForestRegressor(labelCol="total_amount", seed=42)

# Create a pipeline with the defined stages
pipeline = Pipeline(stages=[vector_assembler, rf])

# Fit the model on the training data
model = pipeline.fit(train_data)


In [0]:
# Specify the path to save the model
model_path = "/Machine_Learning/model"

# Save the model to the specified path
model.save(model_path)


In [0]:
# Make predictions on the test data
predictions = model.transform(train_data)

# Evaluate the model using a regression evaluator and calculate RMSE
evaluator = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on Train Data:", rmse)


Root Mean Squared Error (RMSE): 107.74179680830962


In [0]:
# Make predictions on the test data
predictions = model.transform(test_data)

# Evaluate the model using a regression evaluator and calculate RMSE
evaluator = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on Test Data:", rmse)


Root Mean Squared Error (RMSE): 7.64042504062487


In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

# Evaluate the model using Mean Absolute Error (MAE)
evaluator_mae = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(predictions)
print("Mean Absolute Error (MAE):", mae)

# Evaluate the model using R-squared (R2)
evaluator_r2 = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions)
print("R-squared (R2):", r2)





Mean Absolute Error (MAE): 2.732453388553179
R-squared (R2): 0.019652836091647807
